In [2]:
import numpy as np

First, we'll read in the files created by the Fortran HF code. Things are a little messy thanks to the way I stored things in the past, but they are standard numpy arrays after this cell. Take note of the ordering and suggest a more agreeable choice if you've got an opinion.

In [9]:
restartfile = '../HF/wf_numpy.bin'

intchunk = np.dtype(np.int32).itemsize
realchunk = np.dtype(np.float64).itemsize
h = np.fromfile(restartfile, dtype=np.float64, count=1, offset=0)[0]
nbox = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+0*intchunk))[0]
lmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+1*intchunk))[0]
nmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+2*intchunk))[0]
print("hbox = ", h)
print("nbox = ",nbox)
print("lmax = ",lmax)
print("nmax = ",nmax)

#sortstates, sortenergies, wfr,

sortstates = np.fromfile(restartfile, dtype=np.int32, count=nmax*3*2, offset=(realchunk+3*intchunk)).reshape(2,3,nmax)
sortenergies = np.fromfile(restartfile, dtype=np.float64, count=nmax*2, offset=(realchunk+3*intchunk+nmax*3*2*intchunk)).reshape(2,nmax)
wf = np.fromfile(restartfile, dtype=np.float64, count=2*2*(lmax+1)*lmax*(nbox+1), offset=(realchunk+3*intchunk+nmax*3*2*intchunk+sortenergies.nbytes)).reshape(2,2,(lmax+1),lmax,(nbox+1))


hbox =  0.01
nbox =  2000
lmax =  8
nmax =  28


In [5]:
def spin(spin):
    if(spin==1): return -0.5
    return 0.5

def make_input(params):
    in_template = "../HF/in.template"
    f1 = open(in_template, 'r')
    
    print(f1.read())
    int_string = "&interaction t0= "+str(params[0])+",x0=0.77600,t1=484.2300,x1=-0.31700,t2=-556.6900,x2=-1.0,t3=13757.0,x3=1.2630,w0=125.0,sig=6.0,icoul=1,icm=1,j2terms=.true."
    print(int_string)

make_input([2.0])
npart = 0
for i in range(nmax):
    if(sortstates[0,0,i]>0):
        nfill = int(2*(sortstates[0,1,i]+spin(sortstates[0,2,i])) + 1)
        if (sortstates[0,1,i] == 0): nfill = 2
        npart += nfill
        print(npart)


&io printwf=F,printdens=F,restartwf=T /
&box nbox=1000,h=0.02 /
&params r0=1.27,conv=1E-8,hbar22m=20.73553,itermax=200,xmax=0.2 /
&nucleus np=20,nn=28,lmax=8 /
!&interaction t0= -2483.4500,x0=0.77600,t1=484.2300,x1=-0.31700,t2=-556.6900,x2=-1.0,t3=13757.0,x3=1.2630,w0=125.0,sig=6.0,icoul=1,icm=1,j2terms=.true. / ! SLy5
!&interaction t0= -1132.4,x0=0.0,t1=0.0,x1=0.0,t2=0.0,x2=0.0,t3=23610.40,x3=0.0,w0=0.0,sig=1.0,icoul=1,icm=1,j2terms=.false. / !tot3
!&interaction t0= -1128.75,x0=0.0,t1=395.0,x1=0.0,t2=-95.0,x2=-0.0,t3=14000.0,x3=1.0,w0=120.0,sig=1.0,icoul=1,icm=1,j2terms=.false. / ! SIII
!&interaction t0= -2488.913,x0=0.834,t1=486.818,x1=-0.344,t2=-546.395,x2=-1.0,t3=13777.0,x3=1.354,w0=123.0,sig=6.0,icoul=1,icm=1,j2terms=.false. /!

&interaction t0= 2.0,x0=0.77600,t1=484.2300,x1=-0.31700,t2=-556.6900,x2=-1.0,t3=13757.0,x3=1.2630,w0=125.0,sig=6.0,icoul=1,icm=1,j2terms=.true.


NameError: name 'nmax' is not defined